In [ ]:
__author__ = "Emre Kulah, Arif Gorkem Ozer"
__email__ = "emre.klh@gmail.com, arifgorkemozer@gmail.com"

__copyright__ = """
Copyright 2020, 
Middle East Technical University CENG-796  DEEP GENERATIVE MODELS
"""
__license__ = "MIT"
__version__ = "1.0.0"

This Jupyter notebook contains an implementation of the paper [DualGAN: Unsupervised Dual Learning for Image-to-Image Translation](https://arxiv.org/abs/1704.02510).

**Authors:** Zili Yi, Hao (Richard) Zhang, Ping Tan, Minglun Gong

**Published at:** International Conference on Computer Vision, 2017


### Abstract
Conditional Generative Adversarial Networks (GANs)
for cross-domain image-to-image translation have made
much progress recently. Depending
on the task complexity, thousands to millions of labeled image pairs are needed to train a conditional GAN. However,
human labeling is expensive, even impractical, and large
quantities of data may not always be available. Inspired
by dual learning from natural language translation,
we develop a novel dual-GAN mechanism, which enables
image translators to be trained from two sets of unlabeled
images from two domains. In our architecture, the primal
GAN learns to translate images from domain U to those in
domain V , while the dual GAN learns to invert the task.
The closed loop made by the primal and dual tasks allows
images from either domain to be translated and then reconstructed. Hence a loss function that accounts for the reconstruction error of images can be used to train the translators. Experiments on multiple image translation tasks with
unlabeled data show considerable performance gain of DualGAN over a single GAN. For some tasks, DualGAN can
even achieve comparable or slightly better results than conditional GAN trained on fully labeled data.


### How to run?

*    By default, this notebook is configured to run on the **test mode**. Test mode allows you to load pre-trained models and get the results according to their implicit values. To run in **training mode**, you should change **"test"** parameter of **"opt"** to **False**.

*    In test mode, pre-trained models are downloaded from a Google Drive directory. Once the pre-trained models are downloaded, they are ready to use. Downloading them once is enough.

*    Hyper-parameters can be modified by changing values of **"opt"** variable in **Configuration** section in this notebook. 

*    Available datasets in this notebook are **"facades" and "maps"**. "facades" dataset is about generating sketches from pictures of facades and vice versa. "maps" dataset is for generating sketches from satelite views and vice versa.

# Importing Modules



In [ ]:
import torchvision.transforms as transforms
import glob
import os
import numpy as np
import torch
import argparse

import itertools
import sys
import time
import torch.nn as nn
import torch.nn.functional as F
import math
import torch.autograd as autograd

from torch.autograd import Variable
from torchvision.models import vgg19
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from torch.nn import init, modules
from torchvision.utils import save_image
from google_drive_downloader import GoogleDriveDownloader as gdd

Cuda setting (in training):

In [ ]:
cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

# Classes For Generator and Discriminator

The main classes used in this implementation are:

1.   UNetModule
2.   Generator
3.   DiscModule
4.   Discriminator





### UNetModule

This class is used to represent partitions of the UNet model described [here.](https://arxiv.org/pdf/1505.04597.pdf) Each UNetModule can contain convolutional network, batch normalization block, activation block, dropout block. Since the **Generator** class has a UNet structure, it has members in type of UNetModule class. 

If UNetModule instance belongs to UNet segments going down, following operations are done while forwarding data to UNetModules below:

1.   Activation block is applied.
2.   Convolutional block is applied.
3.   Batch normalization is applied.
4.   Dropout is applied.

If UNetModule instance belongs to UNet segments going up, following operations are done while forwarding data to UNetModules above:

1.   Activation block is applied.
2.   Convolutional block is applied.
3.   Batch normalization is applied.
4.   Dropout is applied.
5.   Tanh() is applied (only in output layer, at end of the UNet).

Not every segment of UNet has the exact same configurations, some of the operations mentioned above may not be applied to a particular UNetModule.

In [ ]:
class UNetModule(nn.Module):
    def __init__(self, down=True, input_size=3, output_size=3, kernel_size=4,
                 stride=2, padding=1, activation=True, batch_norm=True, dropout=-1.0, output_layer=False, name=""):
        super(UNetModule, self).__init__()

        self.name = name
        self.down = down
        self.conv_block = nn.Conv2d(in_channels=input_size, out_channels=output_size,
                                    kernel_size=kernel_size, stride=stride, padding=padding)
        self.act_block = nn.LeakyReLU(0.2, True)
        self.bn_block = nn.BatchNorm2d(output_size)
        self.do_block = None
        self.output_layer = output_layer

        # conv transpose and relu as default activation for up
        if not down:
            self.conv_block = torch.nn.ConvTranspose2d(in_channels=input_size, out_channels=output_size,
                                                       kernel_size=kernel_size, stride=stride, padding=padding)
            self.act_block = torch.nn.ReLU(True)

        if not activation:
            self.act_block = None
        if not batch_norm:
            self.bn_block = None
        if dropout != -1.0:
            self.do_block = nn.Dropout(dropout)

    def down_forward(self, x):
        # apply batch norm if exists
        if self.bn_block:
            # apply activation if exists
            if self.act_block:
                out = self.bn_block(self.conv_block(self.act_block(x)))
            else:
                out = self.bn_block(self.conv_block(x))
        else:
            # apply activation if exists
            if self.act_block:
                out = self.conv_block(self.act_block(x))
            else:
                out = self.conv_block(x)

        # apply dropout if exists
        if self.do_block:
            out = self.do_block(out)

        return out

    def up_forward(self, x, skip):
        out = self.down_forward(x)
        if self.output_layer:
            out = torch.nn.Tanh()(out)
        else:
            out = torch.cat([out, skip], 1)
        return out

    def forward(self, x, skip=None):
        if self.down:
            return self.down_forward(x)
        else:
            return self.up_forward(x, skip)

### Generator

Generator class represents generator in the implementation. There are 8 UNetModules for down segments and 8 UNetModules for up segments. **forward()** simply calls forward() of each UNetModule (convolution-down and deconvolution-up segments).


In [ ]:
class Generator(torch.nn.Module):
    def __init__(self, in_channel, out_channel, num_filter, dropout):
        super(Generator, self).__init__()

        # encoder
        self.conv1 = UNetModule(input_size=in_channel, output_size=num_filter, batch_norm=False, activation=False,
                                name="Conv1")
        self.conv2 = UNetModule(input_size=num_filter, output_size=num_filter * 2, name="Conv2")
        self.conv3 = UNetModule(input_size=num_filter * 2, output_size=num_filter * 4, name="Conv3")
        self.conv4 = UNetModule(input_size=num_filter * 4, output_size=num_filter * 8, dropout=dropout, name="Conv4")
        self.conv5 = UNetModule(input_size=num_filter * 8, output_size=num_filter * 8, dropout=dropout, name="Conv5")
        self.conv6 = UNetModule(input_size=num_filter * 8, output_size=num_filter * 8, dropout=dropout, name="Conv6")
        self.conv7 = UNetModule(input_size=num_filter * 8, output_size=num_filter * 8, dropout=dropout, name="Conv7")
        self.conv8 = UNetModule(input_size=num_filter * 8, output_size=num_filter * 8, dropout=dropout, name="Conv8")

        # decoder
        self.deconv1 = UNetModule(down=False, input_size=num_filter * 8, output_size=num_filter * 8, dropout=dropout,
                                  name="Deconv1")
        self.deconv2 = UNetModule(down=False, input_size=num_filter * 16, output_size=num_filter * 8, dropout=dropout,
                                  name="Deconv2")
        self.deconv3 = UNetModule(down=False, input_size=num_filter * 16, output_size=num_filter * 8, dropout=dropout,
                                  name="Deconv3")
        self.deconv4 = UNetModule(down=False, input_size=num_filter * 16, output_size=num_filter * 8, name="Deconv4")
        self.deconv5 = UNetModule(down=False, input_size=num_filter * 16, output_size=num_filter * 4, name="Deconv5")
        self.deconv6 = UNetModule(down=False, input_size=num_filter * 8, output_size=num_filter * 2, name="Deconv6")
        self.deconv7 = UNetModule(down=False, input_size=num_filter * 4, output_size=num_filter, name="Deconv7")
        self.deconv8 = UNetModule(down=False, input_size=num_filter * 2, output_size=out_channel, batch_norm=False,
                                  output_layer=True, name="Deconv8")

    def forward(self, x):
        d1 = self.conv1(x)
        d2 = self.conv2(d1)
        d3 = self.conv3(d2)
        d4 = self.conv4(d3)
        d5 = self.conv5(d4)
        d6 = self.conv6(d5)
        d7 = self.conv7(d6)
        d8 = self.conv8(d7)

        u1 = self.deconv1(d8, d7)
        u2 = self.deconv2(u1, d6)
        u3 = self.deconv3(u2, d5)
        u4 = self.deconv4(u3, d4)
        u5 = self.deconv5(u4, d3)
        u6 = self.deconv6(u5, d2)
        u7 = self.deconv7(u6, d1)
        out = self.deconv8(u7)

        return out

    def initialize(self):
        self.apply(initializer)

### DiscModule

This class is used to represent partitions of the discriminator. 

Following operations are done while forwarding data to next DiscModules:

1.   Convolutional block is applied.
2.   Batch normalization block is applied.
3.   Activation block is applied.

Not every layer of discriminator has the exact same configurations, some of the operations mentioned above may not be applied to a particular DiscModule.

In [ ]:
class DiscModule(nn.Module):
    def __init__(self, input_size, output_size, batch_norm=True, stride=2):
        super(DiscModule, self).__init__()

        self.conv_block = nn.Conv2d(in_channels=input_size, out_channels=output_size,
                                    kernel_size=4, stride=stride, padding=1)
        self.act_block = nn.LeakyReLU(0.2, inplace=True)
        self.bn_block = nn.BatchNorm2d(output_size, 0.8)

        if not batch_norm:
            self.bn_block = None

    def forward(self, x):
        if self.bn_block:
            out = self.act_block(self.bn_block(self.conv_block(x)))
        else:
            out = self.act_block(self.conv_block(x))

        return out

### Discriminator

Discriminator class represents discriminator in the implementation. There are 5 DiscModules for 5 layers. **forward()** simply calls forward() of each DiscModule.


In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channel, out_channel, num_filter):
        super(Discriminator, self).__init__()

        self.conv1 = DiscModule(input_size=in_channel, output_size=num_filter, batch_norm=False)
        self.conv2 = DiscModule(input_size=num_filter, output_size=num_filter * 2)
        self.conv3 = DiscModule(input_size=num_filter * 2, output_size=num_filter * 4)
        self.conv4 = DiscModule(num_filter * 4, num_filter * 8, stride=1)
        self.conv5 = DiscModule(num_filter * 8, out_channel, stride=1, batch_norm=False)

    def forward(self, x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
        out = self.conv5(out)

        return out

    def initialize(self):
        self.apply(initializer)

# Image Loader Classes


In [ ]:
class ImageLoader(DataLoader):
    def __init__(self, root, batch_size, image_size, mode="train"):
        super(ImageLoader, self).__init__(ImageDataset(root, image_size=image_size, mode=mode), batch_size=batch_size,
                                          shuffle=True, num_workers=1)

class ImageDataset(Dataset):
    def __init__(self, root, image_size, mode):
        self.transform = transforms.Compose([
            transforms.Resize((image_size, image_size), Image.BICUBIC),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ])
        # Get images from two sides
        self.filesA = sorted(glob.glob(os.path.join(root, mode) + "/A/*.*"))
        self.filesB = sorted(glob.glob(os.path.join(root, mode) + "/B/*.*"))

    def __getitem__(self, index):
        return (
            self.transform(Image.open(self.filesA[index % len(self.filesA)])),
            self.transform(Image.open(self.filesB[index % len(self.filesB)]))
        )

    def __len__(self):
        return len(self.filesA)

# Utils

In [ ]:
def truncated_normal_(tensor_, mean=0.0, std=1.0):
    size = tensor_.shape
    tmp = tensor_.new_empty(size + (4,)).normal_()
    valid = (tmp < 2) & (tmp > -2)
    ind = valid.max(-1, keepdim=True)[1]
    tensor_.data.copy_(tmp.gather(-1, ind).squeeze(-1))
    tensor_.data.mul_(std).add_(mean)
    return tensor_


def initializer(m):
    if isinstance(m, modules.conv.Conv2d):
        # Weight
        truncated_normal_(m.weight, 0.0, 0.02)
        # Bias
        init.constant_(m.bias, 0.0)

    elif isinstance(m, modules.conv.ConvTranspose2d):
        # Weight
        init.normal_(m.weight, 0.0, 0.02)
        # Bias
        init.constant_(m.bias, 0.0)

    elif isinstance(m, modules.batchnorm.BatchNorm2d):
        # Scale
        init.normal_(m.weight, 1.0, 0.02)
        # Center
        init.constant_(m.bias, 0.0)


def apply_gradient(float_tensor, interpolated, interpolates):
    gradients = autograd.grad(
        outputs=interpolated,
        inputs=interpolates,
        grad_outputs=Variable(float_tensor(np.ones(interpolated.shape)), requires_grad=False),
        create_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    return ((gradients.norm(2, dim=1) - 1) ** 2).mean()

# Helpers

This function is used for generating samples:

In [ ]:
def take_sample(loader, float_tensor, G_AtoB, G_BtoA, dataset_name, batch_idx):
    print("Generating samples")
    images = next(iter(loader))
    imagesA = Variable(images[0].type(float_tensor))
    imagesB = Variable(images[1].type(float_tensor))

    fake_b = G_AtoB(imagesA)
    fake_a = G_BtoA(imagesB)
    sample = torch.cat((imagesA, imagesB, fake_b, fake_a))
    save_image(sample, "training_outputs/%s_%s.png" % (dataset_name, batch_idx), nrow=8, normalize=True)

This function is run when the test mode is active:

In [ ]:
def test_mode(dataset_name, G_AtoB, G_BtoA, val_loader):
    # Download and load pre-trained models if not exists
    # facades GAtoB: https://drive.google.com/open?id=1pbO2jL1-X_1_OTWV0ZwcVavMA0i4vfiz
    # facades GBtoA: https://drive.google.com/open?id=1QjIg6RIbSmEmgF7Lzvu1XgfM9_JQRA5X

    # maps GAtoB: https://drive.google.com/file/d/1njSobeoyU576WbgnieQmnCzHpOYvVkf_
    # maps GBtoA: https://drive.google.com/file/d/1oQUaFHZf9dGHvRuLvRl_qtt_ihyJDTCi
    gatob_pretrained_filename = "%s_GAtoB.pth" % dataset_name
    gbtoa_pretrained_filename = "%s_GBtoA.pth" % dataset_name

    if gatob_pretrained_filename in os.listdir("./training_outputs") and gbtoa_pretrained_filename in os.listdir("./training_outputs"):
        print("Found pre-trained models, skipping downloading")
    else:
        print("Downloading pre-trained models")

        if dataset_name == "maps":
            gdd.download_file_from_google_drive(file_id='1njSobeoyU576WbgnieQmnCzHpOYvVkf_', dest_path='./training_outputs/' + gatob_pretrained_filename, unzip=True, overwrite=True)
            gdd.download_file_from_google_drive(file_id='1oQUaFHZf9dGHvRuLvRl_qtt_ihyJDTCi', dest_path='./training_outputs/' + gbtoa_pretrained_filename, unzip=True, overwrite=True)

        elif dataset_name == "facades":
            gdd.download_file_from_google_drive(file_id='1pbO2jL1-X_1_OTWV0ZwcVavMA0i4vfiz', dest_path='./training_outputs/' + gatob_pretrained_filename, unzip=True, overwrite=True)
            gdd.download_file_from_google_drive(file_id='1QjIg6RIbSmEmgF7Lzvu1XgfM9_JQRA5X', dest_path='./training_outputs/' + gbtoa_pretrained_filename, unzip=True, overwrite=True)

        else:
            print("This dataset is not supported.")
            return

    print("Loading pre-trained models")
    G_AtoB.load_state_dict(torch.load("training_outputs/%s_GAtoB.pth" % dataset_name, map_location="cpu"))
    G_BtoA.load_state_dict(torch.load("training_outputs/%s_GBtoA.pth" % dataset_name, map_location="cpu"))
    take_sample(val_loader, FloatTensor, G_AtoB, G_BtoA, dataset_name, batch_idx="all")

    print("See results in training_outputs/%s_all.png" % dataset_name)

This function is run for initializing optimizers for training mode:

In [ ]:
def init_optimizers(G_AtoB, G_BtoA, learning_rate, betas):
    print("Initializing optimizers")
    g_optim = torch.optim.Adam(
        itertools.chain(G_AtoB.parameters(), G_BtoA.parameters()), lr=learning_rate, betas=betas
    )
    da_optim = torch.optim.Adam(D_A.parameters(), lr=learning_rate, betas=betas)
    db_optim = torch.optim.Adam(D_B.parameters(), lr=learning_rate, betas=betas)

    return g_optim, da_optim, db_optim


This function is run for downloading the dataset if the file does not exist:

In [ ]:
def prepare_dataset(folder_location, dataset_name="facades"):
    dataset_filename = "%s.zip" % dataset_name
    destination = "%s/%s" % (folder_location, dataset_filename)

    if dataset_filename in os.listdir(folder_location):
        print("The dataset file already exists in %s, skipping downloading" % destination)
    else:
        print("Downloading the dataset: %s" % dataset_name)

        if dataset_name == "maps":
            google_drive_file_id = "1J-xVnCOcq986gZrvG3INmqtP8htbyOPG"

        else:
            google_drive_file_id = "1__3713Jm0YoXMReU43qwkgz7n6BwOmAW"

        # facades.zip link: https://drive.google.com/open?id=1__3713Jm0YoXMReU43qwkgz7n6BwOmAW
        # maps.zip link: https://drive.google.com/file/d/1J-xVnCOcq986gZrvG3INmqtP8htbyOPG
        gdd.download_file_from_google_drive(file_id=google_drive_file_id, dest_path=destination, unzip=True, overwrite=True)

    print("Dataset '%s' is ready to use." % dataset_name)

# Hyper-Parameters and Training

### Configuration

By default, this notebook is configured to run on the **test mode**. Test mode allows you to load pre-trained models and get the results according to their implicit values. To run in **training mode**, you should change "test" parameter of "opt" to **False**.

In test mode, pre-trained models are downloaded from a Google Drive directory. Once the pre-trained models are downloaded, they are ready to use. Downloading them once is enough.

By default:

*   5 * 10^-5 is used as learning rate
*   L1-loss is used as loss function. If "loss_metric" is not specified, mean squared error loss function is used by default.
*   50 epochs are run in training mode
*   Dataset name is **"facades"** by default. Other dataset alternative is **"maps"**. When "maps" dataset is loaded, **batch_size = 3** is recommended.
*   Default running mode is "test mode"
*   Default number of samples generated at the end is 4.



In [ ]:
opt = {
    "loss_metric": "L1",
    "img_size": 256,
    "filter_size": 64,
    "channels": 3,
    "dropout_rate": 0.5,
    "lr":  0.00005,
    "beta1": 0.5,
    "beta2": 0.900,
    "num_epochs": 50,
    "batch_size": 4,
    "lambda_A": 500.0,
    "lambda_B": 500.0,
    "dataset_name": "facades",
    "n_cpu": 8,
    "n_critic": 1,
    "test": True,
    "sample_count": 4,
    "save_period": 1,
    "log_freq": 10
}

Dataset is downloaded and used from the directory called **"training_inputs"**. Pre-trained models are downloaded and samples are generated to the directory called **"training_outputs"**.

In [ ]:
# make folders for inputs and outputs
os.makedirs("training_outputs", exist_ok=True)
os.makedirs("training_inputs", exist_ok=True)

# configuration params
img_shape = (opt["channels"], opt["img_size"], opt["img_size"])
gamma = 10.

if opt["loss_metric"] == 'L1':
    norm_loss = torch.nn.L1Loss()
else:
    norm_loss = torch.nn.MSELoss()
  
# download the dataset if it does not exist in 'training_inputs' folder
prepare_dataset(folder_location="./training_inputs", dataset_name=opt["dataset_name"])

The dataset file already exists in ./training_inputs/maps.zip, skipping downloading
Dataset 'maps' is ready to use.


### Training or Testing

In [ ]:
# create models
G_AtoB = Generator(opt["channels"], opt["channels"], opt["filter_size"], opt["dropout_rate"])
G_BtoA = Generator(opt["channels"], opt["channels"], opt["filter_size"], opt["dropout_rate"])
D_A = Discriminator(opt["channels"], 1, opt["filter_size"])
D_B = Discriminator(opt["channels"], 1, opt["filter_size"])

if cuda:
    G_AtoB.cuda()
    G_BtoA.cuda()
    D_A.cuda()
    D_B.cuda()
    norm_loss.cuda()

# create ImageLoader classes for training and validation
data_loader = ImageLoader(root="./training_inputs/%s" % opt["dataset_name"], batch_size=opt["batch_size"], image_size=opt["img_size"])
val_loader = ImageLoader(root="./training_inputs/%s" % opt["dataset_name"], batch_size=opt["sample_count"], image_size=opt["img_size"], mode="val")

# if test mode is selected then generate samples
if opt["test"]:
    print("Running mode: TEST")
    test_mode(opt["dataset_name"], G_AtoB, G_BtoA, val_loader)

else:
    print("Running mode: TRAINING")
    g_optim, da_optim, db_optim = init_optimizers(G_AtoB, G_BtoA, opt["lr"], (opt["beta1"], opt["beta2"]))

    total_batch = 0

    for epoch in range(opt["num_epochs"]):
        for batch_idx, batch in enumerate(data_loader):
            da_optim.zero_grad()
            db_optim.zero_grad()

            # Configure input
            imagesA = Variable(batch[0].type(FloatTensor))
            imagesB = Variable(batch[1].type(FloatTensor))

            # Train discriminators in each epoch

            # Generate a batch of images
            fake_A = G_BtoA(imagesB).detach()
            fake_B = G_AtoB(imagesA).detach()
            fake_ABA = G_BtoA(fake_B)
            fake_BAB = G_AtoB(fake_A)

            # wgan-gp training
            epsilon_A = FloatTensor(np.random.random((opt["batch_size"], 1, 1, 1)))
            epsilon_B = FloatTensor(np.random.random((opt["batch_size"], 1, 1, 1)))

            interpolated_image_A = (imagesA.data + epsilon_A * (fake_A.data - imagesA.data)).requires_grad_(True)
            interpolated_image_B = (imagesB.data + epsilon_B * (fake_B.data - imagesB.data)).requires_grad_(True)
            d_interpolated_A = D_A(interpolated_image_A)
            d_interpolated_B = D_B(interpolated_image_B)
            gradA = apply_gradient(FloatTensor, d_interpolated_A, interpolated_image_A)
            gradB = apply_gradient(FloatTensor, d_interpolated_B, interpolated_image_B)

            # - real image score + fake image score + gp
            disc_loss = (torch.mean(D_A(fake_A)) + torch.mean(D_B(fake_B))) - (
                        torch.mean(D_A(imagesA)) + torch.mean(D_B(imagesB))) + gamma * (gradA + gradB)

            disc_loss.backward()
            da_optim.step()
            db_optim.step()

            if batch_idx % opt["n_critic"] == 0:
                # train generator after each n_critic batch loop
                g_optim.zero_grad()

                ga_loss = -torch.mean(D_A(fake_A)) + opt["lambda_A"] * norm_loss(fake_ABA, imagesA)
                gb_loss = -torch.mean(D_B(fake_B)) + opt["lambda_B"] * norm_loss(fake_BAB, imagesB)
                gen_loss = ga_loss + gb_loss

                gen_loss.backward()
                g_optim.step()
            if batch_idx % opt["log_freq"] == 0:
                print("Iteration: %d - Batch: %d - D-Loss: %.5f - G-Loss: %.5f" % (epoch, batch_idx, disc_loss, gen_loss))
                take_sample(val_loader, FloatTensor, G_AtoB, G_BtoA, opt["dataset_name"], total_batch)

            total_batch += 1

        if epoch % opt["save_period"] == 0:
            torch.save(G_AtoB.state_dict(), "training_outputs/%s_GAtoB.pth" % opt["dataset_name"])
            torch.save(G_BtoA.state_dict(), "training_outputs/%s_GBtoA.pth" % opt["dataset_name"])

Running mode: TEST
Unzipping...Downloading 1oQUaFHZf9dGHvRuLvRl_qtt_ihyJDTCi into ./training_outputs/maps_GBtoA.pth... 

/usr/local/lib/python3.6/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1njSobeoyU576WbgnieQmnCzHpOYvVkf_" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


Done.
Unzipping...Loading pre-trained models


/usr/local/lib/python3.6/dist-packages/google_drive_downloader/google_drive_downloader.py:78: UserWarning: Ignoring `unzip` since "1oQUaFHZf9dGHvRuLvRl_qtt_ihyJDTCi" does not look like a valid zip file
  warnings.warn('Ignoring `unzip` since "{}" does not look like a valid zip file'.format(file_id))


Generating samples
See results in training_outputs/maps_all.png


# Result of Pre-trained Generator Models

# Our Results:

Input dataset: "facades"

![result_facades](https://i.hizliresim.com/1xpJVS.png)



Input dataset: "maps"

![result_maps](https://i.hizliresim.com/JEOspo.png)

Evolution of the output (facades):

![evolution_maps](https://i.hizliresim.com/d9U0vL.gif)

Evolution of the output (maps):

![evolution_maps](https://i.hizliresim.com/92Vkhs.gif)

While creating the gifs above, ImageMagick's "convert" command is used to convert outputs of each batch in each epoch:

```
>_ convert -delay 200 -loop 0 maps*.png out.gif
```

# Figure 3 and Figure 8 from the paper:
![figure_3](https://i.hizliresim.com/KCCTIn.png)

![figure_8](https://i.hizliresim.com/4zt3H3.png)



# Difficulties Encountered

*    U-Net shaped network is used in the paper. It's details do not explained. We analyzed the referenced paper first to understand U-net shaped (number of layers, skips etc.). Since it is not complex, it was not take too much time to implement.
*    WGAN-GP is used in official tensorflow implementation as default and it has better performance. However, paper does not contain this approach.
*    Paper briefly mentions the usage and not much details are shared with readers.